In [ ]:
import numpy as np 
import pandas as pd
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV, GridSearchCV, StratifiedKFold, RepeatedStratifiedKFold, RandomizedSearchCV, GridSearchCV

In [ ]:
train_T = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_I = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
test_T = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
test_I = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')

In [ ]:
test_I.columns=train_I.columns

# Encoding

In [ ]:
train_T = train_T.apply(pd.to_numeric, errors='coerce')

In [ ]:
train_I = train_I.apply(pd.to_numeric, errors='coerce')

In [ ]:
test_T = test_T.apply(pd.to_numeric, errors='coerce')

In [ ]:
test_I = test_I.apply(pd.to_numeric, errors='coerce')

In [ ]:
mergedtestset = test_T.merge(test_I, how= 'left').replace([np.inf, -np.inf], np.nan)

In [ ]:
mergedset = train_T.merge(train_I, how= 'left')

In [ ]:
mergedtestset.fillna(0,inplace= True)

In [ ]:
mergedset.fillna(0,inplace= True)

In [ ]:
points = mergedset.loc[:,(mergedset.columns != 'TransactionID') & (mergedset.columns !=  'isFraud')]

In [ ]:
labels = mergedset['isFraud']

In [ ]:
featurelist= points.columns

In [ ]:
points.shape

In [ ]:
labels.shape

# Random/Grid Search

La busqueda de hiperparametros la hice corriendo en una computadora ya que Kaggle se quedaba sin ram

In [ ]:
# params = {
#         'C': [0,1, 0.5, 2]
#         }

In [ ]:
# LR =  LogisticRegression(solver='liblinear',random_state = 1 )

In [ ]:
# folds = 2
# param_comb = 1

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(LR, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(points,labels), verbose=3, random_state=1001 )

# random_search.fit(points, labels)

In [ ]:
#print('\n All results:')
#print(random_search.cv_results_)
# print('\n Best estimator:')
# print(random_search.best_estimator_)
# print('\n Best score:')
# print(random_search.best_score_ )
# print('\n Best hyperparameters:')
# print(random_search.best_params_)

# ¿Cuál es el mejor score de validación obtenido? (¿Cómo conviene obtener el dataset para validar?)

# Genero Train y Test Sets del set de entrenamiento

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(points, labels, test_size=0.5 ,random_state = 1)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
model = LogisticRegression(C=0.5,solver='liblinear',random_state = 1 )

In [ ]:
model.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, model.predict( X_test))

In [ ]:
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

In [ ]:
print(classification_report(y_test, model.predict(X_test)))

## El mejor test de validacion fue: 

### 0.6484813436281216

# Al predecir con este modelo para la competencia, ¿Cúal es el score obtenido? (guardar el csv con predicciones para entregarlo después)

# Genero Prediccion de la competencia

In [ ]:
y_test= model.predict_proba(mergedtestset.loc[:, (mergedtestset.columns != 'TransactionID')])[:,1]
finalpredLR = mergedtestset.drop(featurelist,axis=1)
finalpredLR['isFraud']= y_test
print(finalpredLR.shape)
finalpredLR.to_csv("prediccionLR.csv",index=False)

## El Score obtnido fue: 

### 0.717802

Hubo un score anterior mas alto de 0.745214, pero para ese solo entrene con el set de Transactions ( es decir no con todos los features) 

# ¿Qué features son los más importantes para predecir con el mejor modelo? Graficar.

# Feature Importance

In [ ]:
features = pd.DataFrame(featurelist.values,columns = ['feature'])

In [ ]:
features['importance'] = model.coef_[0]

In [ ]:
features.set_index('feature',inplace= True)

# Importance for Class 1

In [ ]:
features.loc[features.importance > 0,].sort_values(by= 'importance',ascending=True).plot(kind = 'barh',figsize = (10,40), title = 'Feature Importance class 1')                                                                                                    

# Importance for Class 0

In [ ]:
(features.loc[features.importance < 0,]*(-1)).sort_values(by= 'importance',ascending=True).plot(kind = 'barh',figsize = (10,30),title = 'Feature Importance class 0')

# Importance 0

In [ ]:
features.loc[features.importance == 0,]